# Generating Asset Returns using CAPM

We can break down this problem into a few steps:

- Generate market returns from a normal distribution
- Generate asset idiosyncratic standard deviations from normal distributions
- Generate asset betas from a uniform distribution
- Calculate CAPM for each asset to generate asset returns, also drawing the idiosyncratic return from a normal distribution
- Calculate the correlation of the assets' returns

But first, I will go to Excel and add the inputs table, the same that is shown in the problem document.

## Generate Market Returns from a Normal Distribution

We have seen in other exercises that we can use `random.normalvariate` to pull values from a normal distribution.

In [1]:
import random

mean = 0.05
stdev = 0.1

random.normalvariate(mean, stdev)

0.0756517312498459

But we want as many values as we have periods.

In [2]:
num_periods = 5
random_values = []
for i in range(num_periods):
    random_values.append(random.normalvariate(mean, stdev))
random_values

[0.3796607328864173,
 -0.013015586831232898,
 0.18408997588764386,
 -0.004944748887300107,
 0.002372368078570594]

Now just wrap this up in a function.

In [3]:
def n_random_normal(mean, stdev, num_periods):
    random_values = []
    for i in range(num_periods):
        random_values.append(random.normalvariate(mean, stdev))
    return random_values

n_random_normal(10, 20, 3)

[-5.088644029421385, 14.345142631130704, 38.77947296187832]

As discussed in the Dynamic UDFs example, this will make it output horizontal, but I want it vertical in my model. With this change it will become vertical.

In [4]:
def n_random_normal(mean, stdev, num_periods):
    random_values = []
    for i in range(num_periods):
        random_values.append([random.normalvariate(mean, stdev)])
    return random_values

n_random_normal(10, 20, 3)

[[14.325355040128365], [7.050079564419365], [35.199078996533395]]

Now make this an `xlwings` function so we can call it from excel. Add this to your `xlwings` project `.py` file.

Don't forget to `import random` at the top as well. Now go to Excel, import functions, and generate the market returns using this function. You should now have a column with 20 random returns for the market return.

## Generate Asset Idiosyncratic Return Standard Deviations from Normal Distributions

Here we are again generating a certain amount of numbers from a normal distribution. So we can actually use the previous function for this. But in my workbook, it makes sense for these to be laid out in rows rather than columns. So I will modify the function to accept an argument for whether it should output vertically or horizontally.

In [5]:
def n_random_normal(mean, stdev, num_periods, horizontal=False):
    random_values = []
    for i in range(num_periods):
        num = random.normalvariate(mean, stdev)
        if not horizontal:
            num = [num]
        random_values.append(num)
    return random_values

print(f'Vertical: {n_random_normal(5, 10, 3)}')
print(f'Horizontal: {n_random_normal(5, 10, 3, horizontal=True)}')

Vertical: [[-0.2905436309917544], [6.760555049224479], [27.680099072939036]]
Horizontal: [19.842695827720675, -2.3650502442403614, 12.785091125730997]


Now add back the `xlwings` stuff on the top. I'll also switch the return to be `expand='table'` now that it can go either horizontal or vertical.

Now call this function in Excel to generate the assets' idiosyncratic and standard deviations.

## Generate Asset Betas from a Uniform Distribution

We can use `random.uniform` for this purpose. Just like the prior random number generator, we want to be able to pass it a parameter for how many numbers we want to generate.

In [10]:
bot = 0
top = 2

random.uniform(bot, top)

0.6771644268526513

In [11]:
nper = 3
nums = []
for i in range(nper):
    num = random.uniform(bot, top)
    nums.append(num)
nums

[0.8637176079320799, 0.5642825157171247, 1.7213534787353781]

Wrap it up in a function

In [12]:
def n_random_uniform(bot, top, nper):
    nums = []
    for i in range(nper):
        num = random.uniform(bot, top)
        nums.append(num)
    return nums

n_random_uniform(0, 5, 4)

[4.237723232099332, 3.356470161431886, 2.0711376623220987, 2.398709885117479]

With the current layout, it will output horizontally, which is what I want in my workbook. So I'm good to add the `xlwings` stuff.

Now call this function in Excel, passing 0, 2 and the number of assets.

## Calculate CAPM for each Asset to Generate Asset Returns, also Drawing the Idiosyncratic Return from a Normal Distribution

Now for each asset, we want to calculate:

$r_s = r_f + \beta (r_m - r_f) + \epsilon$

We have $r_f$ as an input. We just calculated $\beta$ and $r_m$. All that's left is drawing $\epsilon$ from a normal distribution and calculating the formula.

First let's just work towards a function that calculates CAPM.

In [13]:
risk_free = 0.02
beta = 0.7
market_ret = 0.2
epsilon = 0.03

risk_free + beta * (market_ret - risk_free) + epsilon

0.176

Ok it's working outside a function, now wrap it in a function.

In [15]:
def capm(risk_free, beta, market_ret, epsilon):
    return risk_free + beta * (market_ret - risk_free) + epsilon

capm(0.01, 0.5, 0.07, -0.06)

-0.019999999999999997

So now we have a function to get the stock return from CAPM. But we know that $\epsilon$ is normally distributed with mean 0, and with a standard deviation that's passed in. So we can work this into the function as well.

In [16]:
epsilon_stdev = 0.1
epsilon = random.normalvariate(0, epsilon_stdev)

capm(0.01, 0.5, 0.07, epsilon)

-0.001951486341375465

Now wrap this up in a function.

In [17]:
def capm_auto_epsilon(risk_free, beta, market_ret, epsilon_stdev):
    epsilon = random.normalvariate(0, epsilon_stdev)

    return capm(risk_free, beta, market_ret, epsilon)

capm_auto_epsilon(0.03, 1.5, 0.05, 0.2)

0.31841752642650956

But we will be taking the entire column of market returns, the entire row of betas, and the entire row of idiosyncratic standard deviations as inputs, as we need it to be resizing dynamically. So we need a version of the function that outputs a table of CAPM returns, based on a list of market returns, list of betas, and a number of assets.

In [37]:
import pandas as pd

num_assets = 2
market_rets = [0.07, 0.05, 0.1]
betas = [0.5, 1, 1.5]
epsilon_stdevs = [0.01, -0.02, 0.05]

df = pd.DataFrame()
for i in range(num_assets):
    beta = betas[i]
    epsilon_stdev = epsilon_stdevs[i]
    returns = [capm_auto_epsilon(risk_free, beta, market_ret, epsilon_stdev) for market_ret in market_rets]
    df[f'Asset {i + 1}'] = returns
df

,Asset 1,Asset 2
0,0.046952,0.050850
1,0.041520,0.015203
2,0.056440,0.118249


Now put this in a function.

In [36]:
def multi_capm(risk_free, betas, market_rets, epsilon_stdevs, num_assets):
    df = pd.DataFrame()
    for i in range(num_assets):
        beta = betas[i]
        epsilon_stdev = epsilon_stdevs[i]
        returns = [capm_auto_epsilon(risk_free, beta, market_ret, epsilon_stdev) for market_ret in market_rets]
        df[f'Asset {i + 1}'] = returns
    return df

multi_capm(risk_free, betas, market_rets, epsilon_stdevs, 5)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5
0,0.043436,0.034225,0.049691,0.070866,0.048018
1,0.039799,0.022794,0.035760,0.035616,0.036754
2,0.146770,0.126195,0.109261,0.044841,0.046520


Now we can make this into an xlwings function. Note that we will have to bring over all of the functions we defined into the `.py` file, but only the one we want to call in Excel will get the `xlwings` stuff on it.

Notice that I added `index=False` to the `@xw.ret` because I'm returning a `DataFrame` and I don't care about the index (0, 1, 2...)

Don't forget to add `import pandas as pd` at the top as well. Now we can generate all the asset's returns by calling this function in Excel. We will call it at the top left of where we want the returns

## Calculate Correlations between the Assets 

Luckily it is very easy to get the correlations with `pandas`. Just `df.corr()`. 

Notice that I included `index=False` on `@xw.arg` so that it won't treat the first column of the data as an index.

Just call this function on another sheet, referencing the top left cell of the data. If you see the correlations then you've completed the exercise.